In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import findspark
findspark.init()
findspark.find()
import pyspark

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)
sqlcontext=SQLContext(sc)

In [2]:
waterDf=spark.read.csv("water_potability.csv",header=True,inferSchema=True)
waterDf.show()

+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------+
|                ph|          Hardness|            Solids|       Chloramines|           Sulfate|      Conductivity|    Organic_carbon|   Trihalomethanes|         Turbidity|Potability|
+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------+
|              null| 204.8904554713363|20791.318980747026| 7.300211873184757|368.51644134980336| 564.3086541722439|  10.3797830780847|  86.9909704615088|2.9631353806316407|         0|
|  3.71608007538699|129.42292051494425|18630.057857970347|    6.635245883862|              null| 592.8853591348523|15.180013116357259| 56.32907628451764| 4.500656274942408|         0|
| 8.099124189298397|224.23625939355776|19909.541732292393| 9.275883602694089|   

In [3]:
waterDf.printSchema()

root
 |-- ph: double (nullable = true)
 |-- Hardness: double (nullable = true)
 |-- Solids: double (nullable = true)
 |-- Chloramines: double (nullable = true)
 |-- Sulfate: double (nullable = true)
 |-- Conductivity: double (nullable = true)
 |-- Organic_carbon: double (nullable = true)
 |-- Trihalomethanes: double (nullable = true)
 |-- Turbidity: double (nullable = true)
 |-- Potability: integer (nullable = true)



In [19]:
waterDf.columns

['ph',
 'Hardness',
 'Solids',
 'Chloramines',
 'Sulfate',
 'Conductivity',
 'Organic_carbon',
 'Trihalomethanes',
 'Turbidity',
 'Potability']

In [8]:
waterDf=waterDf.na.drop("any")

In [9]:
waterDf.count()

2011

In [11]:
waterDf.groupBy("Potability").mean().show()

+----------+------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+--------------------+------------------+---------------+
|Potability|           avg(ph)|     avg(Hardness)|       avg(Solids)| avg(Chloramines)|      avg(Sulfate)| avg(Conductivity)|avg(Organic_carbon)|avg(Trihalomethanes)|    avg(Turbidity)|avg(Potability)|
+----------+------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+--------------------+------------------+---------------+
|         1|7.1137908501089075|195.90834062028335|22344.922883169897|7.174394917458124|332.45783193773906|425.00542273928284| 14.294763978633705|   66.58159561081527| 3.991254060069811|            1.0|
|         0|7.0672009894698995|196.00843971539533|21628.535121541096|7.107266894602621| 333.7429278894922| 427.5543417650412| 14.400250029135762|   66.27871178932857|3.9551813104832756|       

In [15]:
def compute_mean(df):
    return waterDf.groupBy("Potability").mean()
    

In [16]:
test_df=compute_mean(waterDf)

In [17]:
test_df.show()

+----------+------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+--------------------+------------------+---------------+
|Potability|           avg(ph)|     avg(Hardness)|       avg(Solids)| avg(Chloramines)|      avg(Sulfate)| avg(Conductivity)|avg(Organic_carbon)|avg(Trihalomethanes)|    avg(Turbidity)|avg(Potability)|
+----------+------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+--------------------+------------------+---------------+
|         1|7.1137908501089075|195.90834062028335|22344.922883169897|7.174394917458124|332.45783193773906|425.00542273928284| 14.294763978633705|   66.58159561081527| 3.991254060069811|            1.0|
|         0|7.0672009894698995|196.00843971539533|21628.535121541096|7.107266894602621| 333.7429278894922| 427.5543417650412| 14.400250029135762|   66.27871178932857|3.9551813104832756|       

In [21]:
waterDf.groupBy("Potability").agg({'ph':'variance',\
 'Hardness':'variance',
 'Solids':'variance',
 'Chloramines':'variance',
 'Sulfate':'variance',
 'Conductivity':'variance',\
 }).show()

+----------+------------------+---------------------+------------------+-----------------+------------------+----------------------+
|Potability| variance(Sulfate)|variance(Chloramines)|variance(Hardness)|     variance(ph)|  variance(Solids)|variance(Conductivity)|
+----------+------------------+---------------------+------------------+-----------------+------------------+----------------------+
|         1|2251.1409510770864|   3.0025803143125214|1246.1709349326022|2.066759308532427|7.90596252369944E7|     6715.963475936866|
|         0|1324.8437751879576|   2.1802784724172284| 943.5735319491541| 2.75263153489773|7.15903603078976E7|     6381.242140183828|
+----------+------------------+---------------------+------------------+-----------------+------------------+----------------------+



In [22]:
def compute_var(df):
    return waterDf.groupBy("Potability").agg({'ph':'variance',\
 'Hardness':'variance',
 'Solids':'variance',
 'Chloramines':'variance',
 'Sulfate':'variance',
 'Conductivity':'variance',\
 })
test_df=compute_var(waterDf)
test_df.show()

+----------+------------------+---------------------+------------------+-----------------+------------------+----------------------+
|Potability| variance(Sulfate)|variance(Chloramines)|variance(Hardness)|     variance(ph)|  variance(Solids)|variance(Conductivity)|
+----------+------------------+---------------------+------------------+-----------------+------------------+----------------------+
|         1|2251.1409510770864|   3.0025803143125214|1246.1709349326022|2.066759308532427|7.90596252369944E7|     6715.963475936866|
|         0|1324.8437751879576|   2.1802784724172284| 943.5735319491541| 2.75263153489773|7.15903603078976E7|     6381.242140183828|
+----------+------------------+---------------------+------------------+-----------------+------------------+----------------------+



In [25]:
totalsize=waterDf.select("Potability").count()
totalsize

2011

In [26]:
pot_1=waterDf.filter(waterDf["Potability"]==1).count()
pot_1

811

In [28]:
potability_yes=pot_1/totalsize
potability_yes

0.4032819492789657

In [29]:
pot_0=waterDf.filter(waterDf["Potability"]==0).count()
pot_0

1200

In [30]:
potability_no=pot_0/totalsize
potability_no

0.5967180507210343